# imports of functions

In [1]:
from utils import *
from os import listdir, makedirs
from os.path import isfile, join, exists
import os

# arguments for future script

In [2]:
granule = 'ATL03_20210715182907_03381203_005_01.h5'
shapefile = '/shapefiles/jakobshavn_small.shp'
gtxs = 'gt1l'

# download the specified granule via NSIDC
...and subset to the provided shapefile / only pull the variables needed

In [3]:
datadir = '/IS2data'

In [4]:
# %%capture celloutput 
c = edc()
download_granule_nsidc(granule, gtxs, shapefile, datadir, c.u, c.p)


  ATL03_20210715182907_03381203_005_01.h5, 5065.14 MB

Input shapefile: /shapefiles/jakobshavn_small.shp
Simplified polygon coordinates based on shapefile input: -49.59098555896507,69.10415896488335,-50.33805587146507,68.42374368072217,-50.01945235584007,68.20452489563448,-48.47038009021507,68.14325568125464,-48.16276290271507,68.66085965388693,-48.12980391834007,69.0963205694427,-49.59098555896507,69.10415896488335

API request URL:
https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL03&producer_granule_id=ATL03_20210715182907_03381203_005_01.h5&version=005&polygon=-49.59098555896507,69.10415896488335,-50.33805587146507,68.42374368072217,-50.01945235584007,68.20452489563448,-48.47038009021507,68.14325568125464,-48.16276290271507,68.66085965388693,-48.12980391834007,69.0963205694427,-49.59098555896507,69.10415896488335&Boundingshape={"type":"FeatureCollection","features":[{"id":"0","type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-49.59098555896507,6

# read in the .h5 subsetted granule file

In [5]:
filelist = [datadir[1:]+'/'+f for f in listdir(datadir[1:]) if isfile(join(datadir[1:], f)) & ('.h5' in f)]
print('\nNumber of processed ATL03 granules to read in: ' + str(len(filelist)))
    
photon_data, bckgrd_data, ancillary = read_atl03(filelist[0], geoid_h=True)
print_granule_stats(photon_data, bckgrd_data, ancillary, outfile='stats.txt')


Number of processed ATL03 granules to read in: 1
  reading in IS2data/processed_ATL03_20210715182907_03381203_005_01.h5
  reading in beam: gt1l  --> done.

*********************************
** GRANULE INFO AND STATISTICS **
*********************************

ATL03_20210715182907_03381203_005_01.h5
RGT: 338
cycle number: 12
spacecraft orientation: forward
beam configuation:
  gt1l : beam 6 (weak)
  gt1r : beam 5 (strong)
  gt2l : beam 4 (weak)
  gt2r : beam 3 (strong)
  gt3l : beam 2 (weak)
  gt3r : beam 1 (strong)

PHOTON DATA SUMMARY FOR BEAM GT1L
          count  nans           min           max
lat     2593192     0  6.817861e+01  6.904634e+01
lon     2593192     0 -4.965467e+01 -4.936601e+01
h       2593192     0 -2.645939e+02  1.594183e+03
dt      2593192     0  1.116091e+08  1.116091e+08
mframe  2593192     0  3.213661e+09  3.213661e+09
qual    2593192     0  0.000000e+00  2.000000e+00
xatc    2593192     0  7.603724e+06  7.701219e+06
geoid   2593192     0  2.883285e+01  3.21276